In [1]:
import tensorflow as tf

In [2]:
def conv_block(inputs, kernel_size, filters, name, strides=(2, 2)):
    
    filters1, filters2, filters3 = filters

    x = tf.layers.conv2d(inputs,filters1, (1, 1), padding="same"
                        ,name="{}conv1".format(name))
    x = tf.layers.batch_normalization(x ,name="{}batch1".format(name))
    x = tf.nn.relu(x ,name="{}relu1".format(name))

    x = tf.layers.conv2d(x,filters2, kernel_size, strides=strides, padding="same"
                         ,name="{}conv2".format(name))
    x = tf.layers.batch_normalization(x ,name="{}batch2".format(name))
    x = tf.nn.relu(x,name="{}relu2".format(name))

    x = tf.layers.conv2d(x,filters3, (1, 1), padding="same"
                          ,name="{}conv3".format(name))
    x = tf.layers.batch_normalization(x ,name="{}batch3".format(name))

    shortcut = tf.layers.conv2d(inputs, filters3, (1, 1), strides=strides,
                               name="{}conv4".format(name))
    shortcut = tf.layers.batch_normalization(shortcut ,name="{}batch4".format(name))

    x = x + shortcut
    x = tf.nn.relu(x,name="{}relu3".format(name))
    return x

In [3]:
def identity_block(inputs, kernel_size, filters, name):
    
    filters1, filters2, filters3 = filters
    
    x = tf.layers.conv2d(inputs,filters1, (1, 1), padding="same" 
                         ,name="{}identiy_conv1".format(name))
    x = tf.layers.batch_normalization(x,name="{}identiy_batch1".format(name))
    x = tf.nn.relu(x,name="{}identiy_relu1".format(name))

    x = tf.layers.conv2d(x,filters2, kernel_size, padding="same"
                          ,name="{}identiy_conv2".format(name))
    x = tf.layers.batch_normalization(x,name="{}identiy_batch2".format(name))
    x = tf.nn.relu(x,name="{}identiy_relu2".format(name))

    x = tf.layers.conv2d(x,filters3, (1, 1), padding="same"
                          ,name="{}identiy_conv3".format(name))
    x = tf.layers.batch_normalization(x,name="{}identiy_batch3".format(name))

    x = x + inputs
    x = tf.nn.relu(x,name="{}identiy_relu3".format(name))
    return x

In [4]:
def create(x, y, n_classes):
    x = tf.layers.conv2d(x, 64, 7, padding="same")
    x = tf.layers.batch_normalization(x)
    x = tf.nn.relu(x)
    x = tf.layers.max_pooling2d(x, (3,3), (2,2))
    
    x = conv_block(x, 3, [64, 64, 256], strides=(1,1), name="first")
    x = identity_block(x, 3, [64, 64, 256], name="first")
    x = identity_block(x, 3, [64, 64, 256], name="second")
    
    x = conv_block(x, 3, [128, 128, 512], name="second")
    for i in range(0,2): x = identity_block(x, 3, [128, 128, 512], name="third{}".format(i))
    x = conv_block(x, 3, [256, 256, 1024], name="third")
    for i in range(0,4): x = identity_block(x, 3, [256, 256, 1024], name="fourth{}".format(i))
    
    
    x = conv_block(x, 3, [512, 512, 2048], strides=(1,1), name="fifth")
    x = identity_block(x, 3, [512, 512, 2048], name="sixth")
    x = identity_block(x, 3, [512, 512, 2048],name="seventh")
    
    x = tf.layers.average_pooling2d(x,(7, 7), (7,7))
    x = tf.reshape(x, [-1, 4 * 4 * 2048])
    x = tf.layers.dense(
                    x,
                    n_classes,
                )
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=x, labels=y))
    optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)
    correct_pred = tf.equal(tf.argmax(x, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    return x, loss, optimizer, accuracy

In [5]:
from Inputs import *
from prep import prepare_data
import numpy as np

In [6]:
tf.reset_default_graph()

In [7]:
train_images, train_labels, valid_images, valid_labels, n_classes = prepare_data('/run/media/backman/yay/dogbreed/sample/')
train_inputs = Inputs(train_images, train_labels, n_classes, batch_size=2, shuffle=True)
valid_inputs = Inputs(valid_images, valid_labels, n_classes, name="valid", batch_size=10)
with tf.device('/cpu:0'):
    iterator_train = train_inputs.generate_iterator()
    iterator_valid = valid_inputs.generate_iterator()

train_batches_per_epoch = int(np.floor(train_inputs.size / 2))
valid_batches_per_epoch = int(np.floor(valid_inputs.size / 2))

Instructions for updating:
Use `tf.data.Dataset.from_tensor_slices()`.
Instructions for updating:
Replace `num_threads=T` with `num_parallel_calls=T`. Replace `output_buffer_size=N` with `ds.prefetch(N)` on the returned dataset.
Instructions for updating:
Replace `num_threads=T` with `num_parallel_calls=T`. Replace `output_buffer_size=N` with `ds.prefetch(N)` on the returned dataset.


In [8]:
x = tf.placeholder(tf.float32, [None, None, None, 3])
y = tf.placeholder(tf.float32, None)
model, loss, optimizer, accuracy = create(x,y, n_classes)

In [9]:
next_train = iterator_train.get_next()
next_valid = iterator_valid.get_next()
sess = tf.Session()

In [ ]:
sess.run(tf.global_variables_initializer())


In [ ]:
for i in range(1000):
    sess.run(iterator_train.initializer)    
    for i in range(train_batches_per_epoch):
        batch_data, batch_labels = sess.run(next_train)
        feed_dict = {x: batch_data, y: batch_labels}
        res = sess.run([optimizer, loss, accuracy], feed_dict=feed_dict)


In [ ]:
print(res)